<a href="https://colab.research.google.com/github/Richards-R/FCC-Book-Recommendation-Engine-using-KNN/blob/main/Richards_R_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-12-29 05:52:12--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  96.3MB/s    in 0.3s    

2023-12-29 05:52:13 (96.3 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
df_books.loc[df_books['author'] =="Anne Rice"]

In [ ]:
df_ratings.sample(10)

In [ ]:
df_ratings.shape

(1149780, 3)

In [ ]:
df_ratings.describe()

In [ ]:
df_ratings.loc[df_ratings['user']==11676]

In [4]:
df_merged = df_ratings.merge(df_books, on='isbn', how='left')

In [ ]:
df_merged.head(3)

In [ ]:
df_merged.shape

(1031175, 5)

In [5]:
df_merged.sample(5)

,user,isbn,rating,title,author
454300,109013,0842359249,7.0,The Strong-Willed Child: Birth Through Adolesc...,James C. Dobson
26548,6251,1569248281,10.0,The Tick: The Naked City,Ben Edlund
668911,162639,0671019759,0.0,Dream Man,Linda Howard
446127,107021,0060928336,0.0,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells
1011058,242445,1576578208,0.0,Please and Thank You (Spider-Man &amp; Friends),Michi Fujimoto


In [30]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1149780 entries, 0 to 1149779
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1149780 non-null  int32  
 1   isbn    1149780 non-null  object 
 2   rating  1149780 non-null  float32
 3   title   1031175 non-null  object 
 4   author  1031174 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 43.9+ MB


In [25]:
#Clean code
#Find uers with lessthan 200 reviews
#Find books with less than 100 reviews
users = df_merged['user'].value_counts()
books = df_merged['isbn'].value_counts()

users = users[users >= 200].index
books = books[books >= 100].index


In [26]:
print(len(users.values))
print(len(books.values))

905
731


In [12]:
#Make a new data frame
df = df_merged.loc[(df_merged['user'].isin(users.values)) & (df_merged['isbn'].isin(books.values))]

In [27]:
df.describe()

,user,rating
count,49781.000000,49781.000000
mean,140993.388562,2.024086
std,80901.415891,3.576591
min,254.000000,0.000000
25%,70415.000000,0.000000
50%,140358.000000,0.000000
75%,213350.000000,3.000000
max,278418.000000,10.000000


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49781 entries, 1456 to 1147441
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    49781 non-null  int32  
 1   isbn    49781 non-null  object 
 2   rating  49781 non-null  float32
 3   title   49517 non-null  object 
 4   author  49517 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 1.9+ MB


In [13]:
print(users, books)

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
             28634,  59727, 268622, 188951, 225595,  83671, 252827,  99955,
             36554,  26883],
           dtype='int64', length=905) Index(['0971880107', '0316666343', '0385504209', '0060928336', '0312195516',
       '044023722X', '0679781587', '0142001740', '067976402X', '0671027360',
       ...
       '0060916508', '067100042X', '0425189864', '0385424736', '0140092323',
       '0786866586', '0449203794', '0425151867', '0312966806', '0425136981'],
      dtype='object', length=731)


In [35]:
df = df.drop_duplicates(subset=['title', 'user'])

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49136 entries, 1456 to 1147441
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    49136 non-null  int32  
 1   isbn    49136 non-null  object 
 2   rating  49136 non-null  float32
 3   title   49136 non-null  object 
 4   author  49136 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 1.9+ MB


In [37]:
df.isna().sum()

user        0
isbn        0
rating      0
title     238
author    238
dtype: int64

In [42]:
df = df.dropna(how='any')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49136 entries, 1456 to 1147441
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    49136 non-null  int32  
 1   isbn    49136 non-null  object 
 2   rating  49136 non-null  float32
 3   title   49136 non-null  object 
 4   author  49136 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 1.9+ MB


In [44]:
df_pivot = df.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
df_pivot.head(5)

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()